# Atualizar:
- Melhorar função de testar similaridade
- Adicionar memória com limite de mensagens no chat
- Melhorar interface Gradio
- Fasttext
Bertscore
Word2vec
Doc2vec
golve
*glove
-BM25

### Importações

In [1]:
import gradio as gr
from gradio.themes import Soft
import chromadb
import nest_asyncio
from glob import glob
from os import path, getenv, makedirs
from sentence_transformers import SentenceTransformer, util
from dotenv import load_dotenv
from llama_parse import LlamaParse
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.core.node_parser import SemanticSplitterNodeParser, SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore


# Implementação Chroma + LLamaIndex

In [2]:
class Config:
    def __init__(self):
        load_dotenv()
        #self.llm_model = getenv('llm_model')
        self.llm_model = "mistralai/Mistral-Nemo-Instruct-2407"
        self.embedding_model = getenv('embedding_model')
        self.api_key = getenv('api_key')
        self.db_path = getenv('db_path')
        self.input_dir = getenv('input_dir')
        self.collection_name = getenv('collection_name')
        self.parser_key = getenv('parser_key')
        self.connection = None

    def connect_db(self):
        try:
            if not path.exists(self.db_path):
                try:
                    makedirs(self.db_path)
                except:
                    raise Exception(f"Não foi possivel crir a pasta  {self.db_path}.")
            
            db_client = chromadb.PersistentClient(path=self.db_path)
            collection = db_client.get_or_create_collection(self.collection_name)
            self.connection = ChromaVectorStore(chroma_collection=collection)
            
            print("Conexão ao ChromaDB estabelecida com sucesso!")
            return self.connection
        except Exception as e:
            raise Exception(f"Erro ao conectar ao banco de dados: {str(e)}")
        

## Uso do LlamaParse + SemanticSplitterNodeParser

In [3]:

class GenarateIndex:
    def __init__(self):
        self.config = Config()
        self.vector_store = self.config.connect_db()
        self.llm_model = self.config.llm_model
        self.embed_model_name = self.config.embedding_model
        self.input_dir = self.config.input_dir
        self.api_key = self.config.api_key
        self.parser_key = self.config.parser_key
        self.nodes = []

    def generate_index(self):
        try:
            print("\nGerando índice...")
            self.embedding_model = HuggingFaceEmbedding(model_name=self.embed_model_name)
            #splitter = SentenceSplitter(chunk_size=512)
            Settings.node_parser = SemanticSplitterNodeParser(buffer_size=2,
                                                              include_metadata = True, 
                                                              breakpoint_percentile_threshold=90,
                                                              include_prev_next_rel = True,
                                                              embed_model=self.embedding_model,
                                                              #splitter=splitter
                                                              )
            
            self.nodes = Settings.node_parser.get_nodes_from_documents(self.documents)
            

            self.index = VectorStoreIndex(
                nodes = self.nodes,
                vector_store=self.vector_store,
                embed_model=self.embedding_model,
                show_progress=True,
            )
            self.index.storage_context.persist( persist_dir = self.config.db_path)
            
            print("Índice gerado com sucesso!\n")
            return self.index
        except Exception as e:
            raise Exception(f"Erro ao gerar índice: {str(e)}")

    async def process_pdfs(self, input_dir):
        nest_asyncio.apply()
        """
        Processa arquivos PDF em um diretório e extrai texto.
        """
        files = glob(f"{input_dir}/*.pdf")

        try:
            parser = LlamaParse(api_key=self.parser_key, result_type="markdown",)
            file_extractor = {".pdf": parser}
            reader= SimpleDirectoryReader(input_files=files, file_extractor=file_extractor)
            self.documents = await reader.aload_data()
            
            print("PDFs processados com sucesso!")

            return self.documents
        except Exception as e:
            raise Exception(f"Erro ao processar PDFs: {str(e)}")
        
    async def execute(self):
        try:
            nest_asyncio.apply()
            await self.process_pdfs(self.input_dir)
            self.generate_index()
            return self.index
        except Exception as e:
            raise Exception(f"Erro ao executar o processo: {str(e)}")
        



# Chatbot

In [4]:

class ChatBot:
    def __init__(self, index):
        self.config = Config()
        self.api_key = self.config.api_key
        self.llm_model = self.config.llm_model
        self.index = index
        

    def initialize_llm(self):
        try:
            self.prompt = (
                    "Essas são suas diretrizes para interagir com os usuários: "
                    "Seu nome é BOOT. "
                    "Você é um assistente virtual projetado para fornecer informações sobre diversos tópicos. "
                    "Explique passo a passo a resposta"
                    "Para perguntas que envolvam números, resultados ou informações específicas, explique o contexto e o passo a passo para a resposta."
                    "Responda exclusivamente em !![]português. "
                    "Caso o usuário inicie a conversa com uma saudação, cumprimento ou pergunte quem você é ou sobre voê, APENAS apresente-se de forma cordial e simpática. "
                    "Responda com base no que sabe e adapte suas respostas de forma a se encaixar perfeitamente na pergunta ou necessidade do usuário. "
                    "!![]Não use os termos 'Resposta:', 'Pergunta:' ou 'Contexto' ao iniciar suas mensagens. "
                    "Seja educado em todas as interações."
                    "Sempre atenda aos pedidos do usuário em !![]português."
                    )
            Settings.llm = HuggingFaceInferenceAPI(model_name=self.llm_model, 
                                                   token=self.api_key, 
                                                   system_prompt=self.prompt,
                                                   generate_kwargs={"temperature": 0.9})

            Settings.num_output = 1000
            Settings.context_window = 5000

            chat_engine = self.index.as_chat_engine(
                chat_mode="condense_question",
                verbose=False
            )

            return chat_engine
            
        except Exception as e:
            raise Exception(f"Erro ao inicializar Modelo: {str(e)}")
        
    def get_similarity(self, query, context, similarity_threshold=0.3):
        model = SentenceTransformer('all-MiniLM-L6-v2')
        """Calcula a similaridade entre a pergunta e o contexto."""
        query_embedding = model.encode(query)
        filtered_context = []

        for node in context.split('||||'):
            node_embedding = model.encode(node)
            similarity = util.cos_sim(query_embedding, node_embedding).item()

            if similarity > similarity_threshold:
                filtered_context.append(node)
                #print(f"Similaridade: {similarity:.2f}")
                #print("---------------------------")

        return "\n".join(filtered_context)
    
        
    def search(self, query):

        try:

            query_engine = self.index.as_query_engine(
                llm=HuggingFaceInferenceAPI(model_name=self.llm_model, token=self.api_key),
                response_mode="tree_summarize",
                similarity_top_k=6,
                verbose=False
            )
            response = query_engine.query(query)
            context = "||||".join([f"Text: {doc.node.text}" for doc in response.source_nodes])

            filtered_context = self.get_similarity(query, context)
            print(f"Contexto: {filtered_context}")
            print(f"Response : {response}")

            return response, filtered_context

        except Exception as e:
            raise Exception(f"Erro ao realizar a busca: {str(e)}")
        
        
    def respond(self, message, history=None):
        """Gera uma resposta baseada no contexto da pesquisa."""
        try:
            
            # Busca o contexto no ChromaDB
            chat_engine = self.initialize_llm()
            response, context = self.search(message)

            message_context = f"Contexto: {context} /n Pergunta: {message}"

            # Gera a resposta usando o motor de chat
            response = chat_engine.chat(message_context)
            print(f"Resposta: {response.response}")
            return response.response
        
        except Exception as e:
            return f"Erro ao processar mensagem: {str(e)}"
        
    def interface(self): 
        theme = Soft(
        primary_hue="blue",  
        secondary_hue="blue",  
        neutral_hue="gray",  #
        text_size=gr.themes.sizes.text_md,  # Tamanho do texto
        spacing_size=gr.themes.sizes.spacing_lg,  # Espaçamento
        radius_size=gr.themes.sizes.radius_lg,  # Cantos arredondados
    )

        with gr.Blocks(theme=theme, css_paths="style/css/style.css") as demo:
            gr.Image(
                value="style/img/magalu-logo-0-1536x1536.png",  
                show_label=False,
                elem_classes=["logo-style"],  
            )

            gr.ChatInterface(
                fn=self.respond,
                title="ChatBot Personalizado",
                type="messages",
                examples=["Olá", "Como vai você?", "Qual é seu nome?", "Até logo"],
                description="Digite sua mensagem abaixo para conversar com o bot",  
            )
        
        return demo.queue().launch(debug=True)
            
        
 



Gerar o índice:

In [5]:
generate_index = GenarateIndex()
index = await generate_index.execute()

Conexão ao ChromaDB estabelecida com sucesso!
Started parsing the file under job_id a9865e6a-b2c6-4937-b66b-db6a75d4914f
Started parsing the file under job_id 6fdaa7cc-aa3d-4e22-8b66-5b18ac0ce0a9
Started parsing the file under job_id 64a64c6d-6c01-46c3-a2f1-a21556f493e6
............PDFs processados com sucesso!

Gerando índice...


Generating embeddings:   0%|          | 0/176 [00:00<?, ?it/s]

Índice gerado com sucesso!



Rodar a interface de chat:

In [ ]:
chatbot = ChatBot(index)
chatbot.interface()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


## Testes de saída

In [ ]:
chatbot.respond("O que você sabe sobre PDD da magalu? fale sobre o assunto")

Contexto: Text: # Divulgação de Resultados

# 2T24

# Receitas da Intermediação Financeira

No 2T24, as receitas da intermediação financeira atingiram R$657,8 milhões, praticamente estáveis em relação ao mesmo trimestre do ano anterior.

# Provisão para Créditos de Liquidação Duvidosa (PDD)

A carteira vencida de 15 dias a 90 dias (NPL 15) representou apenas 3,0% da carteira total em jun/24, uma melhora de 0,4 p.p. em relação a mar/24 e uma melhora de 0,5 p.p. em relação a jun/23.  Metadata: {'file_path': 'data\\MGLU_ER_2T24_POR.pdf', 'file_name': 'MGLU_ER_2T24_POR.pdf', 'file_type': 'application/pdf', 'file_size': 1782333, 'creation_date': '2025-01-10', 'last_modified_date': '2025-01-10'}
Text: O Magalu não teria a escala, a abrangência e resultados mais resilientes sem isso.

As mais de duas dezenas de operações adquiridas, além de outras desenvolvidas do zero como a Magalu Cloud, foram integradas e conectadas e parte delas se transformou em quatro pilares de geração de resultado: o 

' A carteira vencida de 15 dias a 90 dias (NPL 15) representou apenas 3,4% da carteira total em mar/24, uma variação de 0,3 p.p. em relação a dez/23 e uma melhora de 0,3 p.p. em relação a mar/23.'

O PDD (Provisão para Deterioro de Crédito) é um valor que as empresas reservam para cobrir possíveis perdas financeiras devido a inadimplência de seus clientes. No contexto da Magalu, o PDD representa as despesas líquidas de recuperação, que representaram 2,6% da carteira total no 3T24. A empresa tem observado uma tendência positiva na redução dos indicadores de inadimplência nos últimos meses, o que sinaliza uma contribuição favorável das novas safras para o resultado da Luizacred em relação a jun/24.'

 O PDD (Provision for Doubtful Debts) é um termo utilizado para descrever a provisão para dívidas duvidosas, que é uma reserva financeira criada por uma empresa para cobrir potenciais perdas decorrentes de dívidas que não são prováveis de serem pagas. No contexto da Magalu, o PDD representa as despesas líquidas de recuperação, que são as despesas relacionadas à recuperação de dívidas que não foram pagas pelos clientes. No terceiro trimestre de 2024, o PDD líquido representou 2,6% da carteira total da Magalu. A empresa também observou uma tendência positiva na redução dos indicadores de inadimplência nos últimos meses, o que sugere que as novas safras de clientes estão contribuindo favoravelmente para o resultado da empresa.